In [1]:
#%matplotlib inline

import subprocess
import itertools
import numpy as np
import requests
import pytz
import datetime
import netCDF4
from osgeo import gdal
from os import path
from osgeo.gdalconst import *
from tqdm import tqdm
from bs4 import BeautifulSoup


In [2]:
url_catalog = 'https://opendap.deltares.nl/thredds/catalog/opendap/rijkswaterstaat/maasvlakte/catalog.html'
url_base = 'http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/maasvlakte'
ext = 'nc'
urls = []


def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')

    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]


for ncfile in listFD(url_catalog, ext):
    items = ncfile.split('/catalog.html/')
    filename = items[1].split('/')[-1]
    url = url_base + '/' + filename
    urls.append(url)

In [3]:
urls[:]


['http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/maasvlakte/maasvlakteKB117_3938.nc',
 'http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/maasvlakte/maasvlakteKB116_4140.nc',
 'http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/maasvlakte/maasvlakteKB116_3938.nc']

In [4]:
grids = []
for url in tqdm(urls[:]):
    ds = netCDF4.Dataset(url)
    times = netCDF4.num2date(ds.variables['time'][:], ds.variables['time'].units, calendar='julian')
    local = pytz.timezone("Europe/Amsterdam")
    # times = [local.localize(t, is_dst=None).astimezone(pytz.utc) for t in times]
    times = [datetime.datetime.strptime(t.isoformat(), "%Y-%m-%dT%H:%M:%S").replace(tzinfo=pytz.utc) for t in times]
    arrs = []
    z = ds.variables['z'][:]
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]

    grids.append({
        "url": url,
        "x": x,
        "y": y,
        "z": z,
        "times": times
    })
    ds.close()


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.84s/it]


In [5]:
count = len(list(itertools.chain.from_iterable([g['times'] for g in grids])))
count

27

In [6]:
print(grids[0]['z'][0])

[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]]


In [7]:

#cmd
#subprocess.call('gsutil cp '../output/bathymetry_1985_0001.tif' gs://eo-bathymetry-rws/vaklodingen/bathymetry_1985_0001.tif', shell=True)
#ccc=r"dir"
#ccc
#subprocess.call(ccc)

In [8]:
ee_collection_path = 'projects/bathymetry/assets/maasvlakte' #

In [9]:
def run(cmd, shell=False):
    # print(cmd)
    subprocess.call(cmd,shell=shell)

In [10]:
for g in tqdm(grids):
    print(g['times'])

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

[datetime.datetime(2013, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2014, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2015, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2016, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2018, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2019, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2021, 1, 1, 0, 0, tzinfo=<UTC>)]
[datetime.datetime(2013, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2014, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2015, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2016, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2017, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2018, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2019, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2021, 1, 1, 0, 0, tzinfo=<UTC>)]
[datetime.datetime(2013, 1, 1, 0, 0, tzinfo=<UTC>), datetime.datetime(2014, 1, 1

In [11]:
import os

start_index = 0
dirbathy = r'../../output_maasvlakte/'
j = 0
ts = []
if not os.path.exists(dirbathy):
    os.makedirs(dirbathy)
for g in tqdm(grids):
    ncols = len(g['x'])
    nrows = len(g['y'])
    cellsize = g['x'][1] - g['x'][0]
    # taking corners
    xllcorner = np.min(g['x']-10)
    yllcorner = np.min(g['y']-10)
    nodata_value = -32767
    z = g['z']
    #print(z.shape)

    for i, t in enumerate(g['times']):
        ts.append(t)
        if i < start_index:
            i = i + 1
            continue
        j += 1
        filename = 'maasvlakte_' + str(str(t)[:4]) + '_' + str(j).rjust(3, '0')
        filepath = dirbathy  + filename
        filepath_asc = filepath + '.asc'
        filepath_tif = filepath + '.tif'

        zi = z[i]

        with open(filepath_asc, 'w') as f:
            f.write('ncols {0}\n'.format(ncols))
            f.write('nrows {0}\n'.format(nrows))
            f.write('cellsize {0}\n'.format(cellsize))
            f.write('xllcorner {0}\n'.format(xllcorner))
            f.write('yllcorner {0}\n'.format(yllcorner))
            f.write('nodata_value {0}\n'.format(nodata_value))
            for row in range(nrows-1,-1,-1):
                s = ' '.join([str(v) for v in zi[row,]]).replace('--', str(nodata_value))
                f.write(s)
                f.write('\n')

        #cmd = 'gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff {0} {1}'\
        #    .format(filepath_asc, filepath_tif)
        # per tile
        cmd = 'gdal_translate -ot Float32 -a_srs EPSG:28992 -of COG {0} {1}'\
            .format(filepath_asc, filepath_tif)
        run(cmd)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:25<00:00,  8.48s/it]


In [12]:
# merge per year
uts = list(dict.fromkeys(ts)) # unique times
uyears = [int(str(tt)[:4]) for tt in uts] # unique years

for ii, tt in enumerate(uyears):
    filename = 'maasvlakte_' + str(str(tt)[:4])
    filepath = dirbathy + filename
    filepath_tif = [dirbathy+ll for ll in os.listdir(dirbathy) if str(tt) in ll and ll.endswith('.tif')]
    filepath_year_tif = filepath + '.tif'
    
    # per year
    files_to_mosaic = filepath_tif 
    g = gdal.Warp(filepath_year_tif, files_to_mosaic, dstSRS='EPSG:28992', 
                  outputType=gdal.GDT_Float32, format="COG",
                  options=["COMPRESS=LZW", "TILED=YES"])
    g = None 
    
    filepath_gs = 'gs://eo-bathymetry-rws/maasvlakte/' + filename  # temporary file system in storage bucket
    #print(filepath_gs)
    cmd = 'gsutil cp {0} {1}' \
        .format(filepath_year_tif, filepath_gs)
    run(cmd, shell=True)

    filepath_ee = ee_collection_path + '/' + filename
    #print(filepath_ee)
    cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}' \
        .format(filepath_ee, nodata_value, filepath_gs)
    run(cmd, shell=True)

    time_start = int(uts[ii].timestamp() * 1000)
    cmd = 'earthengine asset set --time_start {0} {1}' \
        .format(time_start, filepath_ee)
    run(cmd, shell=True)

    cmd = 'earthengine acl set public {0}' \
        .format(filepath_ee)
    run(cmd, shell=True)


In [13]:
#filepath_ee
#cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}' \
#              .format(filepath_ee, nodata_value, filepath_gs)
#print(cmd)


In [14]:
# following is just for testing.

In [15]:
        filepath_gs = 'gs://eo-bathymetry-rws/maasvlakte/' + filename
        
        #gsutil = 'D:/src/google-cloud-sdk/bin/gsutil.cmd' # relative path is not defined on Windows
        gsutil = 'gsutil'
        cmd = gsutil + ' cp {0} {1}'\
            .format(filepath_tif, filepath_gs)
        run(cmd)
        
        filepath_ee = ee_collection_path + '/' + filename        
        cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}'\
            .format(filepath_ee, nodata_value, filepath_gs)        
        run(cmd)
        
        time_start = int(grids[0]['times'][0].timestamp() * 1000)
        cmd = 'earthengine asset set --time_start {0} {1}'\
            .format(time_start, filepath_ee)
        run(cmd)

        cmd = 'earthengine acl set public {0}'\
            .format(filepath_ee)
        run(cmd)


FileNotFoundError: [WinError 2] The system cannot find the file specified